In [1]:
import duckdb

MinIO Setup

In [2]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Path yang Digunakan

In [3]:
localCatalogFile = "../metadata/dapodikMetadata.ducklake"
minioNewDataFolder = f"s3://{minioAuth['bucket']}/ducklake/JawaBarat/" # Folder tempat DuckLake menyimpan data barunya
currentMiniofilePath = f"s3://{minioAuth['bucket']}/JawaBarat/dataDKIJakarta.parquet" # LOKASI FILE PARQUET LIVE ANDA
print(currentMiniofilePath)

s3://data-dapodik-kemendikdasmen/JawaBarat/dataDKIJakarta.parquet


DuckDB Connection & Install Extension 

In [4]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

Set MinIO Credentials

In [5]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';")

ATTACH ke DuckLake

In [6]:
con.sql(f"ATTACH 'ducklake:{localCatalogFile}' AS jakartaLake (DATA_PATH '{currentMiniofilePath}');")
con.sql("USE jakartaLake;")

### Langkah 2: Memuat Data Live ke DuckLake (Metadata Management)
Kita akan menggunakan perintah CREATE TABLE AS SELECT untuk membaca file Parquet live Anda dan menuliskannya ke Tabel DuckLake baru.

Perintah ini akan melakukan dua hal:
1. Membuat Schema
1. Membaca konten dari MINIO_LIVE_PARQUET (Parquet yang sudah ada).
2. Menulis data tersebut dalam format yang dikelola DuckLake (dengan commit log metadata) ke dalam Data Path DuckLake Anda (MINIO_DATA_PATH).

In [7]:
con.sql("CREATE SCHEMA IF NOT EXISTS staging;")

# --- 2. Daftarkan dan Load Data Live Anda ---
# Anda bisa melakukan TRANSFORMASI di sini jika diperlukan, atau hanya SELECT *
# staging -> skema, dataDKIJakarta -> tabel
loadLiveDataQuery = f"""
    CREATE TABLE staging.dataDKIJakarta AS
    SELECT 
        * -- Ambil semua kolom dari file Parquet live Anda
    FROM 
        read_parquet('{currentMiniofilePath}');
"""

2. Load Data

In [8]:
con.sql(loadLiveDataQuery)
print(f"\n✅ Data Parquet Live dari {currentMiniofilePath} berhasil dimuat dan terdaftar sebagai 'curated.data_pangan_managed' di DuckLake.")
print(f"File data fisik telah disalin dan sekarang dikelola di folder: {minioNewDataFolder}")


✅ Data Parquet Live dari s3://data-dapodik-kemendikdasmen/JawaBarat/dataDKIJakarta.parquet berhasil dimuat dan terdaftar sebagai 'curated.data_pangan_managed' di DuckLake.
File data fisik telah disalin dan sekarang dikelola di folder: s3://data-dapodik-kemendikdasmen/ducklake/JawaBarat/


3. Verifikasi Snapshot Awal

In [9]:
print("\n--- Riwayat Snapshot Awal ---")
con.sql("SELECT * FROM ducklake_snapshots('bekasiLake', 'staging.dataBekasi');").df()


--- Riwayat Snapshot Awal ---


BinderException: Binder Error: No function matches the given name and argument types 'ducklake_snapshots(VARCHAR, VARCHAR)'. You might need to add explicit type casts.
	Candidate functions:
	ducklake_snapshots(VARCHAR)


In [ ]:
con.close()